In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import find_dotenv, load_dotenv
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent



In [ ]:
load_dotenv(find_dotenv())

In [ ]:
llm = AzureChatOpenAI(model="gpt-4o",
                  temperature = 1,
                  api_version="2023-03-15-preview",
                  verbose=True)

In [ ]:
llm.invoke("Hi")

In [ ]:
template = """
You are puzzle solver. Given a {grid_size}x{grid_size} grid, use PuLP to solve the puzzle.
You should solve the puzzle for the constraints:
1. Each row sum should be 1
2. Each column sum should be 1
3. Each number region sum should be 1
4. Parwise sum of all adjacent cells, row wise, column wise and diagonally should be <= 1

Form these constraints using PuLP and solve them for Binary category since all variables can be either 0 or 1.
Dont give any explanation, execute the code and give me the final answer as 2d array.
Grid:
{grid}
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(template)

In [ ]:
prompt = PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}")""")

In [ ]:
message = prompt_template.invoke({"grid_size": 7, "grid": 
"""[
['1', '1', '1', '2', '2', '2', '3', '3'],
['1', '4', '1', '4', '2', '3', '3', '3'],
['1', '4', '4', '4', '2', '5', '5', '5'],
['1', '4', '4', '4', '2', '6', '6', '6'],
['1', '1', '4', '2', '2', '6', '4', '6'],
['1', '1', '4', '4', '4', '4', '4', '6'],
['1', '7', '4', '4', '4', '4', '4', '6'],
['7', '7', '4', '8', '8', '8', '4', '6']
]"""})

In [ ]:
llm.invoke(message)

In [ ]:
@tool
def execute_python_tool(code):
    """
    A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.
    """
    python_repl = PythonREPLTool()
    return python_repl.invoke(code)

In [ ]:
tools = [execute_python_tool]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke(message)

In [ ]:
agent = create_react_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
message

In [ ]:
agent_executor.invoke({"input": message})

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are puzzle solver. Given a {grid_size}x{grid_size} grid, use PuLP to solve the puzzle.
You should solve the puzzle for the constraints:
1. Each row sum should be 1
2. Each column sum should be 1
3. Each number region sum should be 1
4. Parwise sum of all adjacent cells, row wise, column wise and diagonally should be <= 1

Form these constraints using PuLP and solve them for Binary category since all variables can be either 0 or 1.
Dont give any explanation, execute the code and give me the final answer as 2d array.
Grid:
{grid}
"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
prompt_template